In [ ]:
import random

def fitness(chromosome):
    # Convert binary chromosome to decimal
    x = int(''.join(map(str, chromosome)), 2)
    # Fitness function f(x) = x^2
    return x ** 2

def decimal_to_binary(decimal, length):
    return list(map(int, bin(decimal)[2:].zfill(length)))

def generate_population_from_input(decimal_input, chromosome_length):
    decimal_numbers = map(int, decimal_input.split())
    population = [decimal_to_binary(decimal_number, chromosome_length) for decimal_number in decimal_numbers]
    return population

def calculate_selection_metrics(population, fitnesses):
    total_fitness = sum(fitnesses)
    probabilities = [f / total_fitness for f in fitnesses]
    expected_counts = [p * len(population) for p in probabilities]
    actual_counts = [round(count) for count in expected_counts]

    return probabilities, expected_counts, actual_counts

def select_parents(population, actual_counts):
    selected_parents = []
    for index, count in enumerate(actual_counts):
        selected_parents.extend([population[index]] * count)
    random.shuffle(selected_parents)  # Shuffle to ensure randomness
    return selected_parents

def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    offspring1 = parent1[:point] + parent2[point:]
    offspring2 = parent2[:point] + parent1[point:]
    return offspring1, offspring2

def mutate(chromosome, mutation_rate):
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            chromosome[i] = 1 - chromosome[i]  # Flip the bit
    return chromosome

# Parameters
population_size = int(input("Enter population size: "))
num_generations = int(input("Enter number of generations: "))
mutation_rate = 0.01

# User input for initial population in decimal
decimal_input = input("Enter initial population as space-separated decimal numbers: ")
chromosome_length = max(len(bin(int(d))) - 2 for d in decimal_input.split())

population = generate_population_from_input(decimal_input, chromosome_length)

# Main Genetic Algorithm Loop
best_fitnesses = []
for generation in range(num_generations):
    fitnesses = [fitness(chromosome) for chromosome in population]

    # Calculate selection metrics
    probabilities, expected_counts, actual_counts = calculate_selection_metrics(population, fitnesses)

    # Select parents based on actual counts
    mating_pool = select_parents(population, actual_counts)

    new_population = []
    for i in range(0, len(mating_pool), 2):
        parent1 = mating_pool[i]
        parent2 = mating_pool[i + 1] if i + 1 < len(mating_pool) else mating_pool[0]  # Wrap around for odd counts
        offspring1, offspring2 = crossover(parent1, parent2)

        # Calculate fitness after crossover
        fitness_offspring1 = fitness(offspring1)
        fitness_offspring2 = fitness(offspring2)

        print(f"Generation {generation}, Offspring Fitness after Crossover: {fitness_offspring1}, {fitness_offspring2}")

        # Mutate offspring
        mutated_offspring1 = mutate(offspring1, mutation_rate)
        mutated_offspring2 = mutate(offspring2, mutation_rate)

        # Calculate fitness after mutation
        fitness_mutated_offspring1 = fitness(mutated_offspring1)
        fitness_mutated_offspring2 = fitness(mutated_offspring2)

        new_population.append(mutated_offspring1)
        new_population.append(mutated_offspring2)

    population = new_population
    best_fitness = max(fitnesses)
    best_fitnesses.append(best_fitness)
    print(f"Generation {generation}: Best Fitness = {best_fitness}")

# Final results
final_fitnesses = [fitness(chromosome) for chromosome in population]
print("Final Population (Binary):", population)
print("Final Fitnesses:", final_fitnesses)

Enter population size: 4
Enter number of generations: 1
Enter initial population as space-separated decimal numbers: 12 25 5 19
Generation 0, Offspring Fitness after Crossover: 289, 729
Generation 0, Offspring Fitness after Crossover: 625, 361
Generation 0: Best Fitness = 625
Final Population (Binary): [[1, 0, 0, 0, 1], [1, 1, 0, 1, 1], [1, 1, 0, 0, 1], [1, 0, 0, 1, 1]]
Final Fitnesses: [289, 729, 625, 361]


In [ ]:
import random

# Step 1: Define fitness function (x^2)
def fitness(x):
    return x ** 2

# Step 2: Initialize population
def create_population(size, lower_bound, upper_bound):
    return [random.randint(lower_bound, upper_bound) for _ in range(size)]

# Step 3: Convert integer to 5-bit binary
def to_binary(x):
    return format(x, '05b')

# Step 4: Convert 5-bit binary back to integer
def from_binary(binary_str):
    return int(binary_str, 2)

# Step 5: Calculate the sum, average, and maximum of x^2 for the population
def calculate_statistics(population):
    fitness_values = [fitness(x) for x in population]
    total_sum = sum(fitness_values)
    average = total_sum / len(population)
    maximum = max(fitness_values)
    return fitness_values, total_sum, average, maximum

# Step 6: Selection using roulette wheel (fitness proportional)
def selection(population, fitness_values):
    total_fitness = sum(fitness_values)
    selection_probs = [fit / total_fitness for fit in fitness_values]

    # Roulette wheel selection
    selected_population = random.choices(population, weights=selection_probs, k=len(population))

    return selected_population

# Step 7: Perform crossover on pairs of individuals
def crossover(population):
    random.shuffle(population)
    new_population = []
    for i in range(0, len(population) - 1, 2):
        parent1, parent2 = population[i], population[i + 1]
        crossover_point = random.randint(1, 4)  # 5-bit crossover
        parent1_bin, parent2_bin = to_binary(parent1), to_binary(parent2)
        offspring1_bin = parent1_bin[:crossover_point] + parent2_bin[crossover_point:]
        offspring2_bin = parent2_bin[:crossover_point] + parent1_bin[crossover_point:]
        new_population.append(from_binary(offspring1_bin))
        new_population.append(from_binary(offspring2_bin))
    return new_population

# Step 8: Perform mutation on population (flip one bit)
def mutate(population, mutation_rate=0.05):
    mutated_population = []
    for individual in population:
        if random.random() < mutation_rate:
            individual_bin = list(to_binary(individual))
            mutation_point = random.randint(0, 4)
            individual_bin[mutation_point] = '1' if individual_bin[mutation_point] == '0' else '0'
            mutated_population.append(from_binary(''.join(individual_bin)))
        else:
            mutated_population.append(individual)
    return mutated_population

# Step 9: Genetic algorithm with elitism and biased towards 31
def genetic_algorithm(population_size, generations, lower_bound, upper_bound, mutation_rate):
    population = create_population(population_size, lower_bound, upper_bound)

    # Keep the best individual in each generation (elitism)
    best_solution = None

    for generation in range(generations):
        print(f"Generation {generation + 1}")

        # Step 4: Calculate fitness and statistics
        fitness_values, total_sum, average_fitness, max_fitness = calculate_statistics(population)
        print(f"Sum: {total_sum}, Average: {average_fitness}, Max: {max_fitness}")

        # If we find the maximum fitness, stop
        if max_fitness == 961:
            print(f"Maximum found: x = 31, x^2 = 961")
            return population

        # Step 5: Selection using fitness proportional selection (roulette wheel)
        selected_population = selection(population, fitness_values)

        # Step 6: Elitism - keep the best individual
        best_individual_idx = fitness_values.index(max(fitness_values))
        best_solution = population[best_individual_idx]  # Keep track of the best solution
        new_population = [best_solution]

        # Step 7: Crossover to create new population
        new_population.extend(crossover(selected_population))

        # Step 8: Mutate the population
        new_population = mutate(new_population, mutation_rate)

        # Recalculate fitness
        fitness_values, total_sum, average_fitness, max_fitness = calculate_statistics(new_population)
        print(f"New Population - Sum: {total_sum}, Average: {average_fitness}, Max: {max_fitness}")

        # Step 9: Update population for the next generation
        population = new_population[:population_size]

    # Return the best solution found
    return population

# Parameters
population_size = 4
generations = 50  # Increased generations
lower_bound = 0
upper_bound = 31
mutation_rate = 0.1

# Run the genetic algorithm
best_solution = genetic_algorithm(population_size, generations, lower_bound, upper_bound, mutation_rate)
print(f"Best solution found: {best_solution} with x^2 = {max([fitness(x) for x in best_solution])}")

Generation 1
Sum: 494, Average: 123.5, Max: 484
New Population - Sum: 2305, Average: 461.0, Max: 529
Generation 2
Sum: 1776, Average: 444.0, Max: 484
New Population - Sum: 2260, Average: 452.0, Max: 484
Generation 3
Sum: 1776, Average: 444.0, Max: 484
New Population - Sum: 2836, Average: 567.2, Max: 900
Generation 4
Sum: 2352, Average: 588.0, Max: 900
New Population - Sum: 3028, Average: 605.6, Max: 900
Generation 5
Sum: 2544, Average: 636.0, Max: 900
New Population - Sum: 3444, Average: 688.8, Max: 900
Generation 6
Sum: 2960, Average: 740.0, Max: 900
New Population - Sum: 3860, Average: 772.0, Max: 900
Generation 7
Sum: 2960, Average: 740.0, Max: 900
New Population - Sum: 4129, Average: 825.8, Max: 900
Generation 8
Sum: 3229, Average: 807.25, Max: 900
New Population - Sum: 3790, Average: 758.0, Max: 961
Generation 9
Sum: 3306, Average: 826.5, Max: 961
Maximum found: x = 31, x^2 = 961
Best solution found: [30, 31, 22, 31] with x^2 = 961
